# Experiment 5:

- Our base sketch transformer architecture + class embedding layer
- Multi-class Conditional Model
- DeltaPenPositionTokenizer

In [18]:
from experiment_dir import set_cwd_project_root

set_cwd_project_root()

In [ ]:
from dataset import QuickDrawDataset
from sketch_tokenizers import DeltaPenPositionTokenizer
from models import SketchTransformerConditional
from runner import SketchTrainer, sample
from prepare_data import stroke_to_bezier_single, clean_svg

label_names = ["cake", "butterfly", "flower", "mug", "sea turtle"]
dataset = QuickDrawDataset(label_names=label_names, download=True)
tokenizer = DeltaPenPositionTokenizer(bins=32)

model = SketchTransformerConditional(
    vocab_size=len(tokenizer.vocab),
    d_model=512,
    nhead=8,
    num_layers=8,
    max_len=200,
    num_classes=len(label_names),
)

training_config = {
    "batch_size": 128,
    "num_epochs": 15,
    "learning_rate": 1e-4,
    "log_dir": "logs/sketch_transformer_experiment_6",
    "splits": [0.85, 0.075, 0.075],
    # "use_padding_mask": True,
    "checkpoint_path": "logs/sketch_transformer_experiment_6/20251116_002631/model_3.pt",
}

trainer = SketchTrainer(model, dataset, tokenizer, training_config)

Loading QuickDrawDataset: 5it [00:00, 294.27it/s]
c:\Users\matth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Tokenizing dataset: 100%|██████████| 5/5 [00:00<00:00, 665.55it/s]


Resumed training from checkpoint: logs/sketch_transformer_experiment_6/20251116_002631/model_3.pt


Initial Eval: 100%|██████████| 482/482 [00:02<00:00, 185.03it/s]


In [21]:
trainer.train_mixed(training_config["num_epochs"])

Epoch 4/15 [val]: 100%|██████████| 482/482 [00:22<00:00, 21.50it/s]


Epoch 4 | Train Loss: 0.9401 | Val Loss: 0.9207


Epoch 5/15 [val]: 100%|██████████| 482/482 [00:22<00:00, 21.85it/s]


Epoch 5 | Train Loss: 0.9275 | Val Loss: 0.9118


Epoch 6/15 [val]: 100%|██████████| 482/482 [00:22<00:00, 21.87it/s]


...

Epoch 13 | Train Loss: 0.8916 | Val Loss: 0.8870


Epoch 14/15 [val]: 100%|██████████| 482/482 [00:21<00:00, 22.06it/s]


Epoch 14 | Train Loss: 0.8895 | Val Loss: 0.8861


Epoch 15/15 [val]: 100%|██████████| 482/482 [00:22<00:00, 21.59it/s]


Epoch 15 | Train Loss: 0.8882 | Val Loss: 0.8849


In [22]:
generations_inline = ""
generations = []

for j, label_name in enumerate(label_names):
    for i in range(5):
        generated = sample(
            model=trainer.model,
            start_tokens=[trainer.tokenizer.vocab["START"]],
            temperature=0.8,
            top_k=20,
            top_p=0.7,
            greedy=False,
            eos_id=trainer.tokenizer.vocab["END"],
            class_label=j,
        )
        decoded_sketch = tokenizer.decode(generated, stroke_width=0.3)
        decoded_sketch = stroke_to_bezier_single(decoded_sketch)
        decoded_sketch = clean_svg(decoded_sketch)

        generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated {label_name}</b><br>{decoded_sketch}</div>'
        generations.append((generated, decoded_sketch))


from IPython.display import HTML, display

display(HTML(generations_inline))